In [1]:
import sys
sys.path.append('../RecSysRep/')

In [10]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted_7 = ld.getICMselected('7')
ICM_all = sps.hstack([ICM1, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8, seed=1999)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13646 ( 0.0%) Users that have less than 1 test interactions


In [13]:
import os

ofp = "../models_temp/Similarity_Hybrid/"

models_to_combine_best = {
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' : {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_7': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'icm_3': {'shrink': 2211, 'topK': 188, 'feature_weighting': 'TF-IDF', 'normalize': True},
                            'icm_9': {'shrink': 3568, 'topK': 986, 'feature_weighting': 'TF-IDF', 'normalize': True},
                            'icm_all': {"shrink": 5212, "topK": 923, "feature_weighting": "TF-IDF", "normalize": True},
                            'icm_channel': {"shrink": 2000, "topK": 382, "feature_weighting": "TF-IDF", "normalize": False},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'UserKNN' : {'topK': 448, 'similarity': 'cosine', 'shrink': 756, 'normalize': True, 'feature_weighting': 'TF-IDF', 'URM_bias': True},
                         }

In [14]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [15]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid

In [7]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.HybridScores.DifferentStructure import TwoDifferentModelRecommender

In [16]:
recommender1 = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
recommender2 = ItemKNNCBFRecommender(URM_train, ICM_weighted_7)

model_init(recommender1, 'SLIM_pure', models_to_combine_best['SLIMER'])
model_init(recommender2, 'KNNweigh7', models_to_combine_best['icm_7'])

ItemKNNCBFRecommender: ICM Detected 22 ( 0.1%) items with no features.
../models_temp/Similarity_Hybrid/SLIM_pure.zip
Model found!
SLIMElasticNetRecommender: Loading model from file '../models_temp/Similarity_Hybrid/SLIM_pure'
SLIMElasticNetRecommender: Loading complete
../models_temp/Similarity_Hybrid/KNNweigh7.zip
Model found!
ItemKNNCBFRecommender: Loading model from file '../models_temp/Similarity_Hybrid/KNNweigh7'
ItemKNNCBFRecommender: Loading complete


In [86]:
import numpy as np

n_users = URM_train.shape[0]

new_URM = recommender1._compute_item_score(np.arange(n_users))

In [70]:
print(new_URM)

[[2.15507564e-04 3.11180414e-03 2.93949945e-03 ... 4.03999398e-03
  3.58504243e-04 0.00000000e+00]
 [3.23430798e-03 8.85142479e-03 1.36553431e-02 ... 2.88256466e-01
  3.80629068e-03 1.68456323e-02]
 [1.02747139e-03 7.03300466e-04 2.17231762e-04 ... 3.20245582e-03
  2.36753398e-03 1.53539539e-03]
 ...
 [1.85854686e-03 3.68309557e-03 5.49264019e-04 ... 2.64083818e-02
  1.41071994e-03 1.32901631e-02]
 [1.85873895e-03 3.25692957e-03 0.00000000e+00 ... 2.08801609e-02
  6.78419950e-04 1.19669745e-02]
 [1.43016130e-03 2.40411572e-02 2.62265839e-03 ... 5.98555654e-02
  1.99082028e-03 3.77968070e-03]]


In [59]:
mask_URM = new_URM
mask_URM[mask_URM>0] = 1


233731200.0


In [65]:
print(np.sum(mask_URM, axis=1))
print(mask_URM.shape)

[17205. 18056. 15200. ... 17367. 16607. 17769.]
(13650, 18059)


In [91]:
thresh_URM = new_URM

for user in np.arange(n_users):
    ranking_list, scores = recommender1.recommend(user, cutoff = 100, return_scores = True)
    threshold = scores[0][ranking_list[-1]]
    thresh_URM[user][thresh_URM[user] < threshold] = 0
    
print(thresh_URM)
print(new_URM)

KeyboardInterrupt: 

In [88]:
new_URM_th = sps.csr_matrix(thresh_URM)

In [89]:
new_URM_th.data

array([0.1788429 , 0.28848505, 0.17421623, ..., 0.39439464, 0.5722633 ,
       0.25529942], dtype=float32)

In [90]:
recommender3 = MultiThreadSLIM_SLIMElasticNetRecommender(new_URM_th)
recommender3.fit(**models_to_combine_best['SLIMER'])

SLIMElasticNetRecommender: URM Detected 12892 (71.4%) items with no interactions.


  0%|                                                                                        | 0/18059 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
r_d, _ = evaluator_validation.evaluateRecommender(recommender3)
print(r_d.loc[10]['MAP'])
